In [3]:
from huggingface_hub import login

login()

In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, AutoModelForCausalLM, BitsAndBytesConfig
from tokenization_small100 import SMALL100Tokenizer

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

PackageNotFoundError: No package metadata was found for bitsandbytes

In [4]:
tokenizer = AutoTokenizer.from_pretrained("crumb/fake-gpt-j-17m")
model = AutoModelForCausalLM.from_pretrained("crumb/fake-gpt-j-17m")

tokenizer.pad_token = tokenizer.eos_token

In [5]:
tokens = tokenizer("Who are you?", return_tensors="pt")
generated_tokens = model.generate(**tokens)
text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [6]:
text

['Who are you? Ereld oushe owhon ouethe ndce ous; ous']

In [1]:
import pandas as pd

parallel_corpora = pd.read_csv("parallel_corpora.csv")

In [2]:
parallel_corpora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2636017 entries, 0 to 2636016
Data columns (total 8 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Unnamed: 0      int64 
 1   language1       object
 2   language2       object
 3   book            object
 4   chapter         int64 
 5   verse           object
 6   language1_text  object
 7   language2_text  object
dtypes: int64(2), object(6)
memory usage: 160.9+ MB


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="en-to-tl",
    learning_rate=6e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    stra
)

In [ ]:
from transformers import Trainer
import datasets
import numpy as np

import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # If logits are None (some eval configurations), return empty dict
    if logits is None:
        return {}
    # Convert logits to predicted token ids. Logits may be (batch, seq, vocab)
    preds = np.argmax(logits, axis=-1)
    labels = np.array(labels)
    preds = np.array(preds)
    # Mask out label padding (we use -100 for padding labels). Only keep positions where label != -100
    mask = labels != -100
    if mask.sum() == 0:
        return {}
    preds_flat = preds[mask]
    labels_flat = labels[mask]
    return metric.compute(predictions=preds_flat.astype(np.int32), references=labels_flat.astype(np.int32))

corpora = parallel_corpora[
    (parallel_corpora['language1'] == 'english') 
     & (parallel_corpora['language2'] == 'filipino')
][['language1_text', 'language2_text']].sample(frac=0.066, random_state=42)


# Split the dataframe into train / eval using indices (avoid DataFrame.isin misuse)
train_df = corpora.sample(frac=0.9, random_state=42)
eval_df = corpora.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

# Convert pandas DataFrames to `datasets.Dataset` objects expected by Trainer
train_dataset = datasets.Dataset.from_pandas(train_df)
eval_dataset = datasets.Dataset.from_pandas(eval_df)

# Preprocess: tokenize source and target text columns into model inputs and labels
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['language1_text']
    targets = examples['language2_text']
    # Tokenize inputs
    model_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=max_input_length)
    # Tokenize targets (labels) - use the tokenizer's target mode if available
    try:
        with tokenizer.as_target_tokenizer():
            labels_tokenized = tokenizer(targets, padding='max_length', truncation=True, max_length=max_target_length)
    except AttributeError:
        labels_tokenized = tokenizer(targets, padding='max_length', truncation=True, max_length=max_target_length)
    labels = labels_tokenized['input_ids']
    # Replace pad token ids in labels with -100 so they are ignored by the loss and masked during metrics
    pad_id = getattr(tokenizer, 'pad_token_id', None) or getattr(tokenizer, 'eos_token_id', None)
    if pad_id is not None:
        labels = [[(tok if tok != pad_id else -100) for tok in seq] for seq in labels]
    model_inputs['labels'] = labels
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['language1_text', 'language2_text'])
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=['language1_text', 'language2_text'])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Map:   0%|          | 0/1733 [00:00<?, ? examples/s]

c:\Users\rek\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/193 [00:00<?, ? examples/s]

C:\Users\rek\AppData\Local\Temp\ipykernel_1756\2271472808.py:67: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [4]:
corpora = parallel_corpora[
    (parallel_corpora['language1'] == 'english') 
     & (parallel_corpora['language2'] == 'filipino')
][['language1_text', 'language2_text']]#.sample(frac=0.066, random_state=42)

corpora.to_csv("en-to-tl-corpora.csv")



In [10]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,5.440617,0.035207


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=55, training_loss=6.3848388671875, metrics={'train_runtime': 165.8153, 'train_samples_per_second': 10.451, 'train_steps_per_second': 0.332, 'total_flos': 6307439443968.0, 'train_loss': 6.3848388671875, 'epoch': 1.0})

In [ ]:
generated_tokens = trainer.model.generate(**tokens)